In [1]:
import os
from dotenv import load_dotenv
from account import Account, load_accounts, load_mock_accounts
from action import load_actions, store_actions
from executor import MockExecutor, Executor, exec_next_action

load_dotenv()

CUSTOMER_API_KEY = os.getenv("CUSTOMER_API_KEY")
CUSTOMER_API_KEY_SECRET = os.getenv("CUSTOMER_API_KEY_SECRET")
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

soc_account, dev_account = load_mock_accounts()

{'token_type': 'bearer', 'expires_in': 7200, 'access_token': 'R1A5QjBfMFpyZjRoOE4xR2tESDJ3cGxURnFvdUtvRmUyTXZrWjhkMnFnNXlnOjE2ODMwNTU4ODc3MTE6MTowOmF0OjE', 'scope': ['follows.read', 'offline.access', 'tweet.write', 'like.write', 'users.read', 'tweet.read'], 'refresh_token': 'WlJwSjNnQ0taZ29fcUdNenk3SHlrb0hZYzA2TzM2VW02Slk4YlgxSlJSTGs5OjE2ODMwNTU4ODc3MTE6MToxOnJ0OjE', 'expires_at': 1683063152.425092}
{'token_type': 'bearer', 'expires_in': 7200, 'access_token': 'WjV0OFVac1ljQ3RsSHpoQUMxOWtNbmNYbjF1Q3BObXhnaVlRdWd1MG9FS1pYOjE2ODMwNTU4ODg0OTM6MToxOmF0OjE', 'scope': ['follows.read', 'offline.access', 'tweet.write', 'like.write', 'users.read', 'tweet.read'], 'refresh_token': 'ZUJQcENjRlBSWVJBeXhEQnBMOElnSTBhcC1wM3QyREIzaVltdVZRcVdjNVZTOjE2ODMwNTU4ODg0OTM6MTowOnJ0OjE', 'expires_at': 1683063153.1905978}


In [34]:
actions = load_actions(soc_account, dev_account)
executor = Executor()
actions = exec_next_action(executor, actions)
store_actions(actions)

RTing 0xWeb3Beast 1653482981116334083 effa2021ae066bf36e4fdd20d47eab6891ad89449831086d007bc628e24c4c5c


Forbidden: 403 Forbidden
When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.

In [45]:
import pandas
import tweepy

path = "content/cpa.txt"
contents = list()
current: str = ""
current_id: int = 0
with open(path, "r") as file:
    while True:
        line = file.readline()
        if not line:
            if len(current) > 0:
                contents.append(current)
                current = None
            break
        if line[0:3] == "#ID":
            current_id = line[3:]
            print(line.strip(), current_id.strip())
        else:
            current




#ID001 001
#ID002 002
#ID003 003
#ID004 004
#ID005 005
#ID006 006
#ID007 007
#ID008 008
#ID009 009
#ID010 010
#ID011 011
#ID012 012
#ID013 013
#ID014 014
#ID015 015
#ID016 016
#ID017 017
#ID018 018
#ID019 019
#ID020 020
#ID021 021
#ID022 022
#ID023 023
#ID024 024
#ID025 025
#ID026 026
#ID027 027
#ID028 028
#ID029 029
#ID030 030
